<a href="https://colab.research.google.com/github/RicoStaedeli/NLP2025_CQG/blob/main/Training/3_Training_1_SFT_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SFT 1 Training to generate critical questions
In this training we try to use the generated scores for the evaluated SocratiQ dataset.

## Setup
First we define some constant values and also install all needed libraries



### Installation

In [1]:
!pip install -qqq -U transformers datasets accelerate peft trl bitsandbytes wandb --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [2]:
import gc
import os
import torch
import wandb
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from trl import SFTConfig, SFTTrainer, setup_chat_format
import logging

### Colab
This part is only relevant when using the notebook in google colab

In [3]:
from google.colab import userdata, drive

In [4]:
drive.mount('/content/drive')
token = userdata.get('GITHUB')

Mounted at /content/drive


Clone GitHub Repository to directly push generated files

In [5]:
repo_url = f"https://{token}@github.com/RicoStaedeli/NLP2025_CQG.git"

!git clone {repo_url}

Cloning into 'NLP2025_CQG'...
remote: Enumerating objects: 1704, done.
remote: Counting objects: 100% (352/352), done.
remote: Compressing objects: 100% (220/220), done.
remote: Total 1704 (delta 256), reused 174 (delta 129), pack-reused 1352 (from 2)
Receiving objects: 100% (1704/1704), 75.05 MiB | 23.61 MiB/s, done.
Resolving deltas: 100% (986/986), done.


### Path Variables and Logger

In [6]:
################################################################################
#######################   STATIC VARIABLES      ################################
################################################################################

TRAINING_NUMBER = 7
BASE_MODEL_REPO = "meta-llama/Llama-3.1-8B-Instruct"
MODEL_NAME = "Meta-Llama-3.1-8B-Instruct_SFT_1"

################################################################################
#######################   PATH VARIABLES        ################################
################################################################################

train_dataset_path = "/content/NLP2025_CQG/Data/Processed/CQ SFT Dataset.json"

log_base_path = f"/content/NLP2025_CQG/Training/Logs/Traing_{TRAINING_NUMBER}/Tensorboard/"
os.makedirs(log_base_path, exist_ok=True)

log_file_path = f"/content/NLP2025_CQG/Logs/training_{TRAINING_NUMBER}.log"

model_save_path = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Model/{MODEL_NAME}_finetuned/"
os.makedirs(model_save_path, exist_ok=True)

model_lora_adapter_save_path = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Model/{MODEL_NAME}_lora_adapters/"
os.makedirs(model_lora_adapter_save_path, exist_ok=True)

checkpoint_dir = f"/content/drive/MyDrive/HSG/NLP/Project NLP/Training/Training_{TRAINING_NUMBER}/Checkpoints/"
os.makedirs(checkpoint_dir, exist_ok=True)


################################################################################
#######################   LOGGER                ################################
################################################################################

# Setup logger manually
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Create file handler (only if not already added)
if not logger.handlers:
    fh = logging.FileHandler(log_file_path)
    fh.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)
    logger.addHandler(fh)

# Detect device
device = torch.device(
    "mps" if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available()
    else "cpu"
)

In [7]:
logger.info("--------  Start with Training  -------------")
logger.info(f'Device selected: {device}')
logger.info(f'Model: {MODEL_NAME}')
logger.info(f'Training number: {TRAINING_NUMBER}')

INFO:__main__:--------  Start with Training  -------------
INFO:__main__:Device selected: cuda
INFO:__main__:Model: Meta-Llama-3.1-8B-Instruct_SFT_1
INFO:__main__:Training number: 7


## Training Parameters

In [8]:
# Defined in the secrets tab in Google Colab
wb_token = userdata.get('wandb')
wandb.login(key=wb_token)
torch_dtype = torch.bfloat16

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ricostaedeli (ricostaedeli-hsg) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_REPO)

# Reset chat_template if already set
if tokenizer.chat_template is not None:
    tokenizer.chat_template = None

# Load model
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_REPO,
    quantization_config=bnb_config,
    device_map="auto"
)
model, tokenizer = setup_chat_format(model, tokenizer)
model = prepare_model_for_kbit_training(model)

## Load and preprocess dataset
The raw dataset [SocratiQ](https://github.com/NUS-IDS/eacl23_soqg/tree/main) has a label at the begining of the context. We have to remove that and also tokenize the input for the model training.

In [ ]:
dataset = load_dataset('json', data_files=train_dataset_path)

In [11]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'schema'],
        num_rows: 3426
    })
})


In [12]:
schemas_template = {
"CauseToEffect": """'Cause to Effect' with the examples:
How strong is the generalisation that if <eventA> then <eventB>?
Are there other factors in this particular case that could have interfered with the event of‘<eventB>’?""",

"ExpertOpinion": """'Expert Opinion' with the examples:
Is <expertE> a genuine expert in <domainD>?
Is <eventA> consistent with what other experts in <domainD> say? """,

"Analogy": """'Analogy' with the examples:
Are <C1> and <C2> similar in the respect cited?
Is <eventA> true in <C1>?""",

"FearAppeal": """'Fear Appeal' with the examples:
Is <eventB> bad? Why and to whom is it bad?
Is <eventA> away to prevent <eventB>?"""
}

In [13]:
template_chat_llama = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You generate concise, critical, single-sentence questions for argumentative contexts, matching specified question schemas.<|eot_id|><|start_header_id|>user<|end_header_id|>
### Instruction:
Generate one critical question addressing the provided context. Ensure it matches the schema:
{}

### Context:
{}

Respond with only the generated question.<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
### Response:
{}
"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    contexts        = examples["context"]
    questions       = examples["question"]
    schemas         = examples["schema"]

    texts = []
    for schema_key, context, question in zip(schemas, contexts, questions):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        schema_text = schemas_template.get(schema_key, "Schema Unknown")
        text = template_chat_llama.format(schema_text, context, question) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = dataset.map(formatting_prompts_func, batched = True,)

In [15]:
# Check the structure of the dataset
print(dataset['train'][0]['text'])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You generate concise, critical, single-sentence questions for argumentative contexts, matching specified question schemas.<|eot_id|><|start_header_id|>user<|end_header_id|>
### Instruction:
Generate one critical question addressing the provided context. Ensure it matches the schema:
'Cause to Effect' with the examples:
How strong is the generalisation that if <eventA> then <eventB>?
Are there other factors in this particular case that could have interfered with the event of‘<eventB>’?

### Context:
alternate_viewpoints_perspectives: I'd certainly feel good if the shooter 300 feet above me had to specially, consciously train and practice repetitive trigger techniques on multiple trigger assemblies instead of buying a piece of plastic that does the work for you. I'd feel better if limited magazine sizes were enforced as to make rapid fire less useful for slaughtering a crowd of people at random and give first responders a better

In [16]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'schema', 'text'],
        num_rows: 3426
    })
})


In [17]:
train_dataset = dataset['train']

In [18]:
print(train_dataset)

Dataset({
    features: ['id', 'context', 'question', 'schema', 'text'],
    num_rows: 3426
})


### Train the model

In [ ]:
training_args = SFTConfig(
    per_device_train_batch_size=16,
    per_device_eval_batch_size=66,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    max_length = 2048,
    max_grad_norm=0.3,
    num_train_epochs=1,
    max_steps = 40,
    learning_rate=2e-4,
    bf16=True,
    save_total_limit=3,
    logging_steps=5,
    report_to="wandb",
    output_dir="./results_sft_1/",
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    dataset_text_field="text"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    processing_class=tokenizer,
    args=training_args,
    peft_config=peft_config,
)

In [20]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)

logger.info(f"GPU  Information before Training")
logger.info(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
logger.info(f"{start_gpu_memory} GB of memory reserved.")

INFO:__main__:GPU  Information before Training
INFO:__main__:GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
INFO:__main__:15.57 GB of memory reserved.


In [21]:
trainer_stats = trainer.train()
trainer.train()
trainer.save_model(MODEL_NAME)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
5,3.924800
10,2.208900
15,1.660300
20,1.560000
25,1.497300
30,1.456500
35,1.442800
40,1.465600


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


Step,Training Loss
5,1.432400
10,1.407300
15,1.433100
20,1.443800
25,1.412300
30,1.398500
35,1.383900
40,1.398700


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


In [22]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)

logger.info(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
logger.info(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
logger.info(f"Peak reserved memory = {used_memory} GB.")
logger.info(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
logger.info(f"Peak reserved memory % of max memory = {used_percentage} %.")
logger.info(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

INFO:__main__:458.6218 seconds used for training.
INFO:__main__:7.64 minutes used for training.
INFO:__main__:Peak reserved memory = 37.863 GB.
INFO:__main__:Peak reserved memory for training = 22.293 GB.
INFO:__main__:Peak reserved memory % of max memory = 95.718 %.
INFO:__main__:Peak reserved memory for training % of max memory = 56.357 %.


## Save
Save the finetuned model

In [ ]:
# Flush memory
#del trainer, model
gc.collect()
gc.collect()
torch.cuda.empty_cache()

# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_REPO)
# Reset chat_template if already set
if tokenizer.chat_template is not None:
    tokenizer.chat_template = None

fp16_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_REPO,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
fp16_model, tokenizer = setup_chat_format(fp16_model, tokenizer)

# Merge adapter with base model
model = PeftModel.from_pretrained(fp16_model, MODEL_NAME)
model = model.merge_and_unload()

In [ ]:
model.push_to_hub(MODEL_NAME, use_temp_dir=False)
tokenizer.push_to_hub(MODEL_NAME, use_temp_dir=False)

In [25]:
os.chdir("NLP2025_CQG")
!ls

1_a_Generate_DPO_Dataset.ipynb	      Development
1_Information_preprocessing.md	      Doc
1_Preprocessing.ipynb		      Evaluation
2_Baseline_Generation.ipynb	      INFORMATION.md
2_Information_Baseline_Generation.md  LICENSE
3_Evaluation.ipynb		      Logs
4_Finetuned_Generation.ipynb	      README.md
5_Evaluation_Analytics.ipynb	      Training
Data


In [26]:
!git config --global user.name "Rico Städeli"
!git config --global user.email "rico@yabriga.ch"


commit_message = f"Training Number: {TRAINING_NUMBER}, Training logs in Google Drive."
!git add .
!git commit -m "{commit_message}"
!git push

[main fa77974] Training Number: 7, Training logs in Google Drive.
 1 file changed, 13 insertions(+)
 create mode 100644 Logs/training_7.log
Enumerating objects: 6, done.
Counting objects: 100% (6/6), done.
Delta compression using up to 12 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 714 bytes | 714.00 KiB/s, done.
Total 4 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/RicoStaedeli/NLP2025_CQG.git
   51bedc3..fa77974  main -> main
